In [1]:
class CloseCoroutine(Exception):
    pass

In [9]:
def echo():
    try:
        while True:
            received = yield
            print(received)
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratorExit thrown')
        return 'from a generator exit'

In [10]:
e= echo()
next(e)

In [11]:
e.throw(CloseCoroutine)

StopIteration: coro was closed

In [12]:
e = echo()
next(e)

In [13]:
e.close()

closed method was called/or GeneratorExit thrown


In [14]:
e = echo()
next(e)

In [15]:
e.throw(GeneratorExit)

closed method was called/or GeneratorExit thrown


StopIteration: from a generator exit

In [16]:
def delegator():
    result = yield from echo()
    yield'subgen closed and returned',result
    print('delegato closing ...')

In [17]:
d  = delegator()
next(d)
d.send('hello')

hello


In [18]:
d.throw(CloseCoroutine)

('subgen closed and returned', 'coro was closed')

In [19]:
class IgnoreMe(Exception):
    pass

In [20]:
def echo():
    try:
        while True:
            try:
                received = yield
                print(received)
            except IgnoreMe:
                yield "I'm ignoring you"
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratorExit thrown')
        return 'from a generator exit'
    

In [21]:
d = delegator()
next(d)

In [22]:
d.send('python')

python


In [23]:
result = d.throw(IgnoreMe,1000)

In [24]:
result

"I'm ignoring you"

In [25]:
d.send('rocks')

In [26]:
d.send('rocks')

rocks


In [27]:
d.close()

closed method was called/or GeneratorExit thrown


In [28]:
def echo():
    output = None
    try:
        while True:
            try:
                received = yield output
                print(received)
            except IgnoreMe:
                output =  "I'm ignoring you"
            else:
                output = None
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/or GeneratorExit thrown')
        return 'from a generator exit'

In [29]:
d = delegator()
next(d)

In [30]:
d.send('python')

python


In [31]:
d.throw(IgnoreMe)

"I'm ignoring you"

In [32]:
result = d.send('rocks')

rocks


In [33]:
result

In [34]:
type(result)

NoneType

In [35]:
d.close()

closed method was called/or GeneratorExit thrown


In [36]:
def echo():
    while True:
        received = yield
        print(received)

In [37]:
def delegator():
    yield from echo()

In [38]:
d  = delegator()

In [39]:
next(d)

In [40]:
d.throw(ValueError)

ValueError: 

In [41]:
def delegator():
    try:
        yield from echo()
    except ValueError:
        print('delegator got the value error')
        

In [42]:
d = delegator()
next(d)
d.throw(ValueError)

delegator got the value error


StopIteration: 

In [43]:
def delegator():
    while True:
        try:
            yield from echo()
        except ValueError:
            print('delegator got the value error')

In [44]:
d = delegator()
d.__next__()

In [45]:
d.send('python')

python


In [46]:
d.throw(ValueError)

delegator got the value error


In [47]:
d.send('rocks')

rocks


In [48]:
d.throw(IgnoreMe)

IgnoreMe: 

In [49]:
class WriteAverage(Exception):
    pass

In [52]:
def averager(out_file):
    total = 0
    count = 0
    average = None
    with open(out_file,'w') as f:
        f.write('count,average\n')
        while True:
            try:
                received = yield average
                total+=received
                count+=1
                average = total/count
            except WriteAverage:
                if average is not None:
                    print('saving average to file:', average)
                    f.write(f'{count},{average}\n')

In [53]:
avg = averager('sample.csv')
next(avg)

In [54]:
avg.send(1)
avg.send(2)

1.5

In [55]:
avg.throw(WriteAverage)

saving average to file: 1.5


1.5

In [56]:
avg.send(3)

2.0

In [57]:
avg.send(2)

2.0

In [58]:
avg.throw(WriteAverage)

saving average to file: 2.0


2.0

In [59]:
avg.close()

In [60]:
with open('sample.csv') as f :
    for row in f:
        print(row.strip())

count,average
2,1.5
4,2.0


In [61]:
def delegator():
    yield from averager('sample.csv')

In [62]:
d = delegator()

In [63]:
next(d)

In [64]:
d.send(1)

1.0

In [65]:
d.send(3)

2.0

In [66]:
d.send(10)

4.666666666666667

In [67]:
d.throw(WriteAverage)

saving average to file: 4.666666666666667


4.666666666666667

In [68]:
d.send(4)

4.5

In [69]:
d.throw(WriteAverage)

saving average to file: 4.5


4.5

In [70]:
d.close()

In [73]:
with open('sample.csv')as f :
    for row in f:
        print(row.strip())

count,average
3,4.666666666666667
4,4.5
